In [ ]:
import tensorflow as tf

In [ ]:
IMG_WIDTH  = 28
IMG_HEIGHT = 28
IMG_CHANNELS = 1

IMG_DIR = "./EMNIST"
IMG_LABELS = ['lower_0', 'lower_1', 'lower_2', 'lower_3', 'lower_4',
			  'lower_5', 'lower_6', 'lower_7', 'lower_8', 'lower_9',
			  'lower_a', 'lower_b', 'lower_d', 'lower_e', 'lower_f',
			  'lower_g', 'lower_h', 'lower_n', 'lower_q', 'lower_r',
			  'lower_t', 'upper_A', 'upper_B', 'upper_C', 'upper_D',
			  'upper_E', 'upper_F', 'upper_G', 'upper_H', 'upper_I',
			  'upper_J', 'upper_K', 'upper_L', 'upper_M', 'upper_N',
			  'upper_O', 'upper_P', 'upper_Q', 'upper_R', 'upper_S',
			  'upper_T', 'upper_U', 'upper_V', 'upper_W', 'upper_X',
			  'upper_Y', 'upper_Z']

In [ ]:
# Load in datasets using tensorflow
dataset = tf.keras.preprocessing.image_dataset_from_directory(
	IMG_DIR,
	labels="inferred",
	label_mode="categorical",           
	image_size=(IMG_WIDTH, IMG_HEIGHT),
    color_mode="grayscale",
	class_names=IMG_LABELS
)

In [ ]:
# Raw images are in the range 0..255 and we want to normalize the dataset
def normalize_image(image, label):
	image = tf.cast(image, tf.float32) / 255.0
	return image, label

smooth_images = dataset.map(normalize_image)

In [ ]:
# To make the training move a little faster, there are a few options we tried

# smooth_images = smooth_images.cache()  # This can take up a LOT of memory, so uncomment if you have some available
smooth_images = smooth_images.prefetch(tf.data.AUTOTUNE)

In [ ]:
# Split the dataset into test and train sets 80/20
train_size = int(0.8 * len(smooth_images))

train_set = smooth_images.take(train_size)
val_set   = smooth_images.skip(train_size)

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dropout, Dense
# Create the model

model = tf.keras.models.Sequential()

model.add(Conv2D(
	input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS),
	kernel_size=5,
	filters=8,
	strides=1,
	activation=tf.keras.activations.relu
))

model.add(BatchNormalization())

model.add(MaxPool2D(
	pool_size=(2, 2),
	strides=(2, 2)
))

model.add(Conv2D(
	kernel_size=5,
	filters=16,
	strides=1,
	activation=tf.keras.activations.relu
))

model.add(BatchNormalization())

model.add(MaxPool2D(
	pool_size=(2, 2),
	strides=(2, 2)
))

model.add(Flatten())

model.add(Dense(
	units=200,
	activation=tf.keras.activations.relu
))

model.add(Dropout(0.2))

model.add(Dense(
	units=len(IMG_LABELS),
	activation=tf.keras.activations.softmax,
	kernel_initializer=tf.keras.initializers.VarianceScaling()
))

In [ ]:
# Print a quick summary
model.summary()

In [ ]:
# Compile the model

sgd_optimizer = tf.keras.optimizers.SGD()
model.compile(
	optimizer=sgd_optimizer,
	loss=tf.keras.losses.CategoricalCrossentropy(),
	metrics=['accuracy']
)

In [ ]:
# Training the model

training_history = model.fit(
	train_set,
	epochs=25,
	validation_data=val_set
)

print("Model is done training")

In [ ]:
# Training loss and accuracy
train_loss = training_history.history['loss']
train_accuracy = training_history.history['accuracy']

# Testing loss and accuracy
val_loss = training_history.history.get('val_loss')
val_accuracy = training_history.history.get('val_accuracy')

# Print statistics for each epoch
for epoch in range(len(train_loss)):
    print(f"Epoch {epoch+1}:")
    print(f"  Training Loss: {train_loss[epoch]:.4f}, Training Accuracy: {train_accuracy[epoch]:.4f}")
    if val_loss and val_accuracy:
        print(f"  Validation Loss: {val_loss[epoch]:.4f}, Validation Accuracy: {val_accuracy[epoch]:.4f}")

In [ ]:
# Save the results of training
model.save("nocra_demo_train.keras")